In [29]:
import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score, make_scorer
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn import preprocessing
from sklearn.linear_model import LogisticRegression
from scipy import stats
import pickle
from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, predict_model, finalize_model, save_model, load_model
import os
import warnings
warnings.filterwarnings("ignore")

## Carga 

In [30]:
path = "D:/Users/Asus/Documents/PYTHON PARA APIS/Proyecto Final 1 Corte/"

In [32]:
dataset = pd.read_csv(path + "train.csv",sep = ",",decimal=".")
prueba = pd.read_csv(path + "test.csv",sep = ",",decimal=".")

In [4]:
def load_data():
    dataset = pd.read_csv(path + "train.csv",sep = ",",decimal=".")
    prueba = pd.read_csv(path + "test.csv",sep = ",",decimal=".")

    ct = [ 'Marital status', 'Application mode','Application order','Course','Daytime/evening attendance', 'Nacionality',
      "Mother's occupation","Father's occupation", 'Displaced', 'Educational special needs','Debtor','Tuition fees up to date', 'Gender', 
      'Scholarship holder','International']
    for k in ct:
      dataset[k] = dataset[k].astype("O")
      prueba[k] = prueba[k].astype("O")

    cuantitativas = ['Previous qualification','Previous qualification (grade)', 'Admission grade','Age at enrollment','Unemployment rate','Inflation rate','GDP',
                 "Mother's qualification","Father's qualification",'Curricular units 1st sem (credited)','Curricular units 1st sem (enrolled)',
                'Curricular units 1st sem (evaluations)','Curricular units 1st sem (approved)','Curricular units 1st sem (without evaluations)',
                'Curricular units 2nd sem (credited)','Curricular units 2nd sem (enrolled)','Curricular units 2nd sem (evaluations)',
                'Curricular units 2nd sem (approved)','Curricular units 2nd sem (without evaluations)'] 
    return ct,cuantitativas

In [34]:
dataset

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP,Target
0,0,1,1,1,9238,1,1,126.0,1,1,...,0,6,7,6,12.428571,0,11.1,0.6,2.02,Graduate
1,1,1,17,1,9238,1,1,125.0,1,19,...,0,6,9,0,0.000000,0,11.1,0.6,2.02,Dropout
2,2,1,17,2,9254,1,1,137.0,1,3,...,0,6,0,0,0.000000,0,16.2,0.3,-0.92,Dropout
3,3,1,1,3,9500,1,1,131.0,1,19,...,0,8,11,7,12.820000,0,11.1,0.6,2.02,Enrolled
4,4,1,1,2,9500,1,1,132.0,1,19,...,0,7,12,6,12.933333,0,7.6,2.6,0.32,Graduate
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76513,76513,1,17,1,9254,1,1,121.0,1,19,...,0,6,8,5,10.600000,0,13.9,-0.3,0.79,Graduate
76514,76514,1,1,6,9254,1,1,125.0,1,1,...,0,6,9,6,13.875000,0,9.4,-0.8,-3.12,Graduate
76515,76515,5,17,1,9085,1,1,138.0,1,37,...,0,5,8,5,11.400000,1,9.4,-0.8,-3.12,Enrolled
76516,76516,1,1,3,9070,1,1,136.0,1,38,...,0,6,0,0,0.000000,0,7.6,2.6,0.32,Dropout


In [26]:
prueba

,id,Marital status,Application mode,Application order,Course,Daytime/evening attendance,Previous qualification,Previous qualification (grade),Nacionality,Mother's qualification,...,Curricular units 1st sem (without evaluations),Curricular units 2nd sem (credited),Curricular units 2nd sem (enrolled),Curricular units 2nd sem (evaluations),Curricular units 2nd sem (approved),Curricular units 2nd sem (grade),Curricular units 2nd sem (without evaluations),Unemployment rate,Inflation rate,GDP
0,76518,1,1,1,9500,1,1,141.0,1,3,...,0,0,8,0,0,0.000000,0,13.9,-0.3,0.79
1,76519,1,1,1,9238,1,1,128.0,1,1,...,0,0,6,6,6,13.500000,0,11.1,0.6,2.02
2,76520,1,1,1,9238,1,1,118.0,1,1,...,0,0,6,11,5,11.000000,0,15.5,2.8,-4.06
3,76521,1,44,1,9147,1,39,130.0,1,1,...,0,3,8,14,5,11.000000,0,8.9,1.4,3.51
4,76522,1,39,1,9670,1,1,110.0,1,1,...,0,0,6,9,4,10.666667,2,7.6,2.6,0.32
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
51007,127525,1,1,2,171,1,1,128.0,1,38,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51008,127526,2,39,1,9119,1,19,133.1,1,19,...,0,0,5,5,0,0.000000,0,9.4,-0.8,-3.12
51009,127527,1,1,1,171,1,1,127.0,1,1,...,0,0,0,0,0,0.000000,0,15.5,2.8,-4.06
51010,127528,1,1,3,9773,1,1,132.0,1,19,...,0,0,6,9,3,13.000000,0,7.6,2.6,0.32


In [17]:
# Supongamos que tienes los archivos 'train.csv' y 'test.csv' en el mismo directorio.
dataset_path = 'train.csv'
prueba_path = 'test.csv'

# Crear la instancia con ingeniería de variables (True) y usar el primer modelo (1)
modelo = Entry_ML_Flow(dataset_path, prueba_path, apply_variable_engineering=True, model_choice=1)

# Ejecutar el pipeline ML_FLOW
salida = modelo.ML_FLOW()

# Imprimir la salida
print(salida)


NameError: name 'dataset' is not defined

In [36]:
import pandas as pd
import pickle
from pycaret.classification import setup, compare_models, create_model, tune_model, plot_model, predict_model, finalize_model, save_model, load_model
from sklearn.metrics import accuracy_score
import os

dataset = pd.read_csv(path + 'train.csv')
prueba = pd.read_csv(path + "test.csv")


categoricas = ['Marital status', 'Application mode', 'Application order', 'Course', 'Daytime/evening attendance', 'Nacionality',
               "Mother's occupation", "Father's occupation", 'Displaced', 'Educational special needs', 'Debtor', 'Tuition fees up to date', 'Gender',
               'Scholarship holder', 'International']

cuantitativas = ['Previous qualification', 'Previous qualification (grade)', 'Admission grade', 'Age at enrollment', 'Unemployment rate', 'Inflation rate', 'GDP',
                 "Mother's qualification", "Father's qualification", 'Curricular units 1st sem (credited)', 'Curricular units 1st sem (enrolled)',
                 'Curricular units 1st sem (evaluations)', 'Curricular units 1st sem (approved)', 'Curricular units 1st sem (grade)', 'Curricular units 1st sem (without evaluations)',
                 'Curricular units 2nd sem (credited)', 'Curricular units 2nd sem (enrolled)', 'Curricular units 2nd sem (evaluations)', 'Curricular units 2nd sem (approved)',
                 'Curricular units 2nd sem (grade)', 'Curricular units 2nd sem (without evaluations)']

# ingeniería de variables
def apply_engineering(data):
    # Aquí se podría agregar lógica adicional de transformación de variables
    data_cuan = data[cuantitativas]
    data_cat = data[categoricas]
    return data_cuan, data_cat


def pipeline(dataset, prueba, apply_feature_engineering, model_choice):
    # Si se aplica ingeniería de variables
    if apply_feature_engineering:
        dataset_cuan, dataset_cat = apply_engineering(dataset)
        # Recombinamos las variables transformadas (simulación)
        dataset = pd.concat([dataset_cuan, dataset_cat], axis=1)
    
    # Configuración inicial del experimento en PyCaret
    exp_clf101 = setup(data=dataset,
                       target='Target',
                       session_id=123,
                       train_size=0.7,
                       numeric_features=cuantitativas,
                       categorical_features=categoricas,
                       fix_imbalance=False)

    # Comparar modelos y seleccionar los mejores
    best_model = compare_models(sort='AUC')
    
    # Crear modelos
    model1 = create_model('lightgbm')
    model2 = create_model('xgboost')
    model3 = create_model('gbc')

    # Seleccionar el modelo según el parámetro model_choice
    if model_choice == 1:
        selected_model = model1
        model_name = 'ModeloSI1'
    elif model_choice == 2:
        selected_model = model2
        model_name = 'ModeloSI2'
    elif model_choice == 3:
        selected_model = model3
        model_name = 'ModeloSI3'

    # Guardar el dataset
    with open(path + f'{model_name}.pkl', 'wb') as model_file:
        pickle.dump(dataset, model_file)

    # Parámetros de ajuste
    param_grid_bayesian = {
        'n_estimators': [50, 100, 200],
        'max_depth': [3, 5, 7],
        'min_child_samples': [50, 150, 200]
    }

    # modelo seleccionado
    tuned_model = tune_model(selected_model, custom_grid=param_grid_bayesian, search_library='scikit-optimize', search_algorithm='bayesian', fold=5)

    # Evaluar el modelo
    plot_model(tuned_model, plot='confusion_matrix')

    # Predicciones en conjunto de prueba y entrenamiento
    predictions_test = predict_model(tuned_model)
    predictions_train = predict_model(tuned_model, data=exp_clf101.get_config('X_train'))

    y_train = exp_clf101.get_config('y_train')
    y_test = exp_clf101.get_config('y_test')

    # Calcular precisión
    print("Precisión de entrenamiento:", accuracy_score(y_train, predictions_train["prediction_label"]))
    print("Precisión de prueba:", accuracy_score(y_test, predictions_test["prediction_label"]))

    # Finalizar el modelo
    final_model = finalize_model(tuned_model)

    # Realizar predicciones en nuevos datos (test)
    predictions = predict_model(final_model, data=prueba)

    # Ajustar probabilidad de score
    predictions["sc"] = predictions.apply(lambda row: 1 if row["prediction_score"] < 0.9 else 0, axis=1)
    predictions["sc"].value_counts()

    # Crear archivo para Kaggle
    result = pd.DataFrame({
        'id': prueba["id"],
        'Target': predictions['sc']
    })

    # Guardar archivo de Kaggle
    result.to_csv(path + f'/sample_submission_{model_name}.csv', index=False, sep=",")

    # Guardar el modelo final
    save_model(final_model, path + f'final_{model_name}_model')

    # Cargar el modelo si es necesario
    loaded_model = load_model(path + f'final_{model_name}_model')

    return loaded_model

# Ejecución del pipeline con parámetros
loaded_model = pipeline(dataset, prueba, apply_feature_engineering=True, model_choice=1)


ValueError: Invalid value for the target parameter. Column Target not found in the data.

In [14]:
salida

NameError: name 'salida' is not defined

In [23]:
# Supongamos que tienes los archivos 'train.csv' y 'test.csv' en el mismo directorio.
dataset_path = 'train.csv'
prueba_path = 'test.csv'

# Crear la instancia con ingeniería de variables (True) y usar el primer modelo (1)
modelo = Entry_ML_Flow(dataset_path, prueba_path, apply_variable_engineering=True, model_choice=1)

# Ejecutar el pipeline ML_FLOW
salida = modelo.ML_FLOW()

# Imprimir la salida
print(salida)

{'success': False, 'message': "[Errno 2] No such file or directory: 'train.csv'"}


In [37]:
yhat_train, yhat_test = model_predictions(0.5)
X_train, X_test, y_train, y_test = load_data()

NameError: name 'model_predictions' is not defined

In [38]:
umbral(y_train,yhat_train)

NameError: name 'umbral' is not defined

In [39]:
umbral(y_test,yhat_test)

NameError: name 'umbral' is not defined

In [40]:
u1 = umbral(y_train,yhat_train) #accuracy
u2 = umbral(y_test,yhat_test)
valores = [u1,u2,-np.abs(u1-u2)] #valor absoluto de la resta de los accuracy
comparacion = [80,80,-5]
for i,j in zip(valores,comparacion):
    print(i>=j)

NameError: name 'umbral' is not defined